# AI辅助资治通鉴知识图谱挖掘&构建

In [1]:
import json

In [2]:
book = json.load(open("adapted_book.json", "r"))
book[0]

{'juan_index': 1,
 'juan_title': '资治通鉴第一卷(周纪)',
 'segments': [{'segment_index': 1,
   'segment_start_time': '周纪一 威烈王二十三年（戊寅、前403）',
   'sentences': ['[1]周威烈王姬午初次分封晋国大夫魏斯、赵籍、韩虔为诸侯国君。',
    '臣司马光曰：我知道天子的职责中最重要的是维护礼教，礼教中最重要的是区分地位，区分地位中最重要的是匡正名分。什么是礼教？就是法纪。什么是区分地位？就是君臣有别。什么是名分？就是公、侯、卿、大夫等官爵。',
    '四海之广，亿民之众，都受制于天子一人。尽管是才能超群、智慧绝伦的人，也不能不在天子足下为他奔走服务，这难道不是以礼作为礼纪朝纲的作用吗！所以，天子统率三公，三公督率诸侯国君，诸侯国君节制卿、大夫官员，卿、大夫官员又统治士人百姓。权贵支配贱民，贱民服从权贵。上层指挥下层就好像人的心腹控制四肢行动，树木的根和干支配枝和叶；下层服侍上层就好像人的四肢卫护心腹，树木的枝和叶遮护根和干，这样才能上下层互相保护，从而使国家得到长治久安。所以说，天子的职责没有比维护礼制更重要的了。',
    '周文王演绎排列《易经》，以乾、坤为首位。孔子解释说：“天尊贵，地卑微，阳阴于是确定。由低至高排列有序，贵贱也就各得其位。”这是说君主和臣子之间的上下关系就像天和地一样不能互易。《春秋》一书贬低诸侯，尊崇周王室，尽管周王室的官吏地位不高，在书中排列顺序仍在诸侯国君之上，由此可见孔圣人对于君臣关系的关注。如果不是夏桀、商纣那样的暴虐昏君，对手又遇上商汤、周武王这样的仁德明主，使人民归心、上天赐命的话，君臣之间的名分只能是作臣子的恪守臣节，矢死不渝。所以如果商朝立贤明的微子为国君来取代纣王，成汤创立的商朝就可以永配上天；而吴国如果以仁德的季札做君主，开国之君太伯也可以永享祭祀。然而微子、季札二人宁肯国家灭亡也不愿做君主，实在是因为礼教的大节绝不可因此破坏。所以说，礼教中最重要的就是地位高下的区分。',
    '所谓礼教，在于分辨贵贱，排比亲疏，裁决万物，处理日常事物。没有一定的名位，就不能显扬；没有器物，就不能表现。只有用名位来分别称呼，用器物来分别标志，然后上下才能井然有序。这就是礼教的根本所在。如果名位、器

In [ ]:
!pip3 install --upgrade langchain
!pip3 install -U langchain-community

In [ ]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [28]:
import os

from langchain_deepseek import ChatDeepSeek
from openai import OpenAI

# Create a ChatOpenAI instance
chat_model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    # max_tokens=None,
    timeout=None,
    # max_retries=2,
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    cache=True,
)

# Define a conversation
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"},
]

# Get a response from the model
response = chat_model.invoke(conversation)
response

AIMessage(content='Hello! 😊 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': 'a1015ac8-76a0-4819-bc1f-180aea6ec0c2', 'finish_reason': 'stop', 'logprobs': None}, id='run-36bb8e46-caa0-4374-a1cd-d9afb7bcf7b3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [38]:
extraction_prompt = """
Process Chinese historical text chapters and extract structured entity-relation data. Maintain strict formatting for downstream processing.

# Steps
1. **Entity Identification**  
- Extract all influential figures, organizations, states, and cultural artifacts  
- Record all known aliases from the text  
- Note first chronological appearance with era/year format  
- Preserve original descriptive text verbatim
- Record belonged power if mentioned in original descriptive text
- Generate concise summary synthesizing key attributes  

2. **Relation Mapping**  
- Identify all inter-entity actions: military, political, social  
- Capture named historical events (战/盟/变格式)  
- Record temporal context using era/year format  
- Specify directional relationships (actor → target)  
- Document outcomes when explicitly stated  

# Output Format  
Strict JSON without markdown code blocks:  
```json
{
  "entities": [
    {
      "name": "规范名称",
      "alias": ["文本出现的所有别名"],
      "first_occurrence_time": "纪年格式",
      "original_description_in_book": "原文描述",
      "llm_introduction": "综合分析的实体简介",
      "power": "所属势力派别或None",
    }
  ],
  "relations": [
    {
      "time": "关系或动作发生时间或None",
      "from": "主动方，必须出现在entities中",
      "to": ["所有接收方/相同实体，必须出现在entities中"],
      "action": "具体动作",
      "result": "结果或None", 
      "event_name": "事件名称或None",
      "location": "关系或动作发生的地点或None",
    }
  ]
}
```

# Examples  
Input: "前453年，晋阳之战爆发，智伯联合韩、魏围赵襄子于晋阳。后韩魏倒戈，与赵合灭智氏，三家分晋格局成。"  
Output:
{
  "entities": [
    {
      "name": "智伯",
      "alias": ["智氏"],
      "first_occurrence_time": "前453年",
      "original_description_in_book": "晋阳之战发起方",
      "llm_introduction": "晋国卿大夫，主导晋阳之战最终兵败",
      "power": "智氏家族",
    },
    {
      "name": "三家分晋",
      "alias": [],
      "first_occurrence_time": "前453年", 
      "original_description_in_book": "韩赵魏灭智氏后的政治格局",
      "llm_introduction": "标志春秋转入战国时代的关键事件",
      "power": null,
    }
  ],
  "relations": [
    {
      "time": "前453年",
      "from": "智伯",
      "to": ["赵襄子"],
      "action": "围攻",
      "result": "失败",
      "event_name": "晋阳之战",
      "location": "晋阳",
    },
    {
      "time": "前453年",
      "from": "韩魏",
      "to": ["智伯"], 
      "action": "背叛",
      "result": "智氏灭亡",
      "event_name": null,
      "location": null
    }
  ]
}

# Notes  
- 挖掘**所有实体和关系**
- 关系方向必须反映原文动作方向  
- 事件名称仅当原文明确命名时记录  
- 别名需穷举文本中出现的所有变称  
- 实体简介需保持客观，不添加原文未有的信息
- 关系中的实体必须出现在实体的记录之中，若无，应该将缺失的实体补充至实体记录中
- 评论内容也应包含在关系中，from为评论者实体，to为评论的对象，result为评论内容
"""

In [48]:
!unset http_proxy
!unset https_proxy

In [49]:
chat_model = ChatDeepSeek(
    # model="deepseek-reasoner",
    model="deepseek-chat",
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=0,
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    cache=True,
    disable_streaming=True,
)

# Define a conversation
conversation = [
    {
        "role": "user",
        "content": extraction_prompt
        + "\n\n"
        + json.dumps(book[0], ensure_ascii=False, indent=1),
    },
]

# Get a response from the model
response = chat_model.invoke(conversation)
response

AIMessage(content='```json\n{\n  "entities": [\n    {\n      "name": "周威烈王",\n      "alias": ["姬午"],\n      "first_occurrence_time": "前403年",\n      "original_description_in_book": "周威烈王姬午初次分封晋国大夫魏斯、赵籍、韩虔为诸侯国君。",\n      "llm_introduction": "周朝君主，首次分封晋国大夫为诸侯，标志战国时代开始。",\n      "power": "周朝"\n    },\n    {\n      "name": "司马光",\n      "alias": [],\n      "first_occurrence_time": "前403年",\n      "original_description_in_book": "臣司马光曰：我知道天子的职责中最重要的是维护礼教...",\n      "llm_introduction": "《资治通鉴》作者，以礼教为核心评论周威烈王分封事件。",\n      "power": null\n    },\n    {\n      "name": "魏斯",\n      "alias": ["魏文侯"],\n      "first_occurrence_time": "前403年",\n      "original_description_in_book": "晋国大夫魏斯被分封为诸侯国君",\n      "llm_introduction": "魏国建立者，礼贤下士的政治家，开创魏国霸权。",\n      "power": "魏国"\n    },\n    {\n      "name": "智瑶",\n      "alias": ["智襄子"],\n      "first_occurrence_time": "前403年",\n      "original_description_in_book": "智瑶有超越他人的五项长处...但很不仁厚",\n      "llm_introduction": "晋国卿大夫，才胜于德导致智氏灭亡的关键人物。",\n      "powe

In [52]:
json.loads(response.content.replace('```json', '').replace('```', ''))

{'entities': [{'name': '周威烈王',
   'alias': ['姬午'],
   'first_occurrence_time': '前403年',
   'original_description_in_book': '周威烈王姬午初次分封晋国大夫魏斯、赵籍、韩虔为诸侯国君。',
   'llm_introduction': '周朝君主，首次分封晋国大夫为诸侯，标志战国时代开始。',
   'power': '周朝'},
  {'name': '司马光',
   'alias': [],
   'first_occurrence_time': '前403年',
   'original_description_in_book': '臣司马光曰：我知道天子的职责中最重要的是维护礼教...',
   'llm_introduction': '《资治通鉴》作者，以礼教为核心评论周威烈王分封事件。',
   'power': None},
  {'name': '魏斯',
   'alias': ['魏文侯'],
   'first_occurrence_time': '前403年',
   'original_description_in_book': '晋国大夫魏斯被分封为诸侯国君',
   'llm_introduction': '魏国建立者，礼贤下士的政治家，开创魏国霸权。',
   'power': '魏国'},
  {'name': '智瑶',
   'alias': ['智襄子'],
   'first_occurrence_time': '前403年',
   'original_description_in_book': '智瑶有超越他人的五项长处...但很不仁厚',
   'llm_introduction': '晋国卿大夫，才胜于德导致智氏灭亡的关键人物。',
   'power': '智氏家族'},
  {'name': '三家分晋',
   'alias': [],
   'first_occurrence_time': '前403年',
   'original_description_in_book': '韩赵魏灭智氏后的政治格局',
   'llm_introduction': '春秋转入战国时代的关键事件，晋国被